<a href="https://colab.research.google.com/github/BlackCurrantDS/ATiML-Project/blob/master/LDA_withoutNouns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install pyLDAvis

In [2]:
import nltk; nltk.download('stopwords')
import pandas as pd
import numpy as np
import re
import spacy
import en_core_web_sm
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [36]:
# Import Dataset
books = pd.read_csv("books_withoutNouns.csv")
print(books.genre.unique())
books.head()

['Sea and Adventure' 'Western Stories' 'Love and Romance'
 'Ghost and Horror' 'Humorous and Wit and Satire' 'Literary'
 'Detective and Mystery' 'Christmas Stories']


,Unnamed: 0,Unnamed: 0.1,id,genre,data
0,0,0,0,Sea and Adventure,ago narrative reverse previous hound reelingly...
1,1,1,1,Sea and Adventure,straight reigned high proper great lofty hills...
2,2,2,2,Sea and Adventure,sandy lazily inland great grim bustle temporar...
3,3,3,3,Sea and Adventure,wider great upland main single great real fart...
4,4,4,4,Sea and Adventure,medium frequently comparatively rare high true...


In [5]:
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import nltk
nltk.download('averaged_perceptron_tagger')
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
#last 5
#books['data'] = books['data'].apply(lambda x: str(x)[-100:])
#first 5
books['data'] = books['data'].apply(lambda x: str(x)[:500])
# random 2/3
#books['NData'] = books['NData'].apply(lambda x: x[3000:int(len(x)*1/3)])


In [38]:
# Convert to list
data = books.data.values.tolist()
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

#print(data[:1])

<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: Dep

In [0]:
"""
from nltk.tokenize import RegexpTokenizer
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(data)):
    data[idx] = data[idx].lower()  # Convert to lowercase.
    data[idx] = tokenizer.tokenize(data[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
data = [[token for token in doc if not token.isnumeric()] for doc in data]

# Remove words that are only one character.
data = [[token for token in doc if len(token) > 1] for doc in data]

"""

In [0]:
"""
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
data = [[lemmatizer.lemmatize(token) for token in doc] for doc in data]
"""

In [0]:
"""
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(data, min_count=20)
for idx in range(len(data)):
    for token in bigram[data[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            data[idx].append(token)
"""

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
"""
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(data)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)
"""

In [0]:
"""
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in data]
"""

In [0]:
"""
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))
"""

Number of unique tokens: 236
Number of documents: 40


In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

#print(data_words[:1])

In [41]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['VERB', 'NOUN', 'PPN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [0]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['VERB' , 'PPN'])

#print(data_lemmatized[:1])

In [0]:
"""
# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token
"""

In [0]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [49]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                            alpha='auto',
                                            eta='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [50]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics(5))
doc_lda = lda_model[corpus]

[(3, '0.041*"leave" + 0.041*"hurt" + 0.040*"rag" + 0.040*"fall" + 0.040*"disappear" + 0.040*"annoy" + 0.040*"strike" + 0.040*"realize" + 0.040*"easygoe" + 0.040*"gasp"'), (4, '0.074*"smix" + 0.039*"shipboard" + 0.039*"embark" + 0.039*"unimagine" + 0.039*"unavaile" + 0.039*"feel" + 0.039*"foul" + 0.039*"reverse" + 0.039*"stunt" + 0.039*"blunt"'), (5, '0.032*"admit" + 0.032*"bring" + 0.032*"disengage" + 0.032*"attribute" + 0.032*"forgive" + 0.032*"object" + 0.032*"frighten" + 0.032*"intimate" + 0.032*"mittene" + 0.032*"lewie"'), (2, '0.032*"live" + 0.032*"leave" + 0.032*"close" + 0.032*"shake" + 0.031*"commonplace" + 0.031*"greenmottle" + 0.031*"saunter" + 0.031*"impassive" + 0.031*"unmarrie" + 0.031*"rub"'), (1, '0.051*"close" + 0.027*"smooth" + 0.027*"shake" + 0.027*"represent" + 0.027*"soften" + 0.027*"stumble" + 0.027*"worry" + 0.027*"mean" + 0.027*"clasp" + 0.027*"charm"')]


In [51]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -5.990979644344814

Coherence Score:  0.5547117507790102


In [52]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.149351  0.067661       1        1  18.934696
2     -0.035495 -0.081521       2        1  14.376127
5      0.099520  0.111192       3        1  13.987652
7     -0.000948 -0.002419       4        1  12.233844
6      0.037845 -0.053653       5        1  11.136759
0      0.028965 -0.052668       6        1  10.068611
4      0.001455  0.030562       7        1   9.644231
3      0.018009 -0.019156       8        1   9.618080, topic_info=         Term      Freq     Total Category  logprob  loglift
27       live  2.000000  2.000000  Default  30.0000  30.0000
120      smix  1.000000  1.000000  Default  29.0000  29.0000
32       hurt  1.000000  1.000000  Default  28.0000  28.0000
72      leave  2.000000  2.000000  Default  27.0000  27.0000
25      close  2.000000  2.000000  Default  26.0000  26.0000
..        ...       ...       ...      ...      ...      ...
68    receive  0.059558  1.056441   Topic8  -5.3771  -0.5342
39    respect  0.059526  1.002236   Topic8  -5.3777  -0.4820
121     elsie  0.059557  1.056110   Topic8  -5.3771  -0.5339
94   dislodge  0.059553  1.056365   Topic8  -5.3772  -0.5342
67    pertain  0.059553  1.056439   Topic8  -5.3772  -0.5343

[381 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
109       5  0.997927     abrupt
102       4  0.968249  adjective
35        3  0.946687     admire
59        3  0.946888      admit
69        4  0.968160      amble
...     ...       ...        ...
97        2  0.947109   unmarrie
89        1  0.890465     untrie
53        8  1.019408    upturne
48        1  0.890609      worry
108       4  0.968219      write

[129 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 6, 8, 7, 1, 5, 4])

In [0]:
#Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CalledProcessError: ignored